In [ ]:
%load_ext autoreload

In [ ]:
import altair as alt
import polars as pl

%autoreload
from altair_utils import (
    color_line_yellow_purple,
    line_strokeDash,
)

In [ ]:
df = pl.read_csv(r"Q:\Model Research\downtown_today\accommodation.csv")

In [ ]:
occupancy = df.select("year", "type", "occupancy rate")
supplydemand = (
    df.select("year", "type", "supply (nights)", "demand (nights)")
    .rename({"supply (nights)": "supply", "demand (nights)": "demand"})
    .unpivot(
        index=["year", "type"],
        on=["supply", "demand"],
        variable_name="supply/demand",
        value_name="nights",
    )
)

In [ ]:
occupancy_chart = (
    alt.Chart(occupancy.with_columns(date=pl.date(pl.col("year"), 1, 1)).sort("year"))
    .mark_line()
    .encode(
        x=alt.X("date", title="year"),
        y=alt.Y("occupancy rate", title="SF accommodation occupancy rate").axis(
            format="%"
        ),
        # color=color_value_transit_teal,
        color=alt.Color("type").scale(
            # domain=["supply", "demand"],
            range=color_line_yellow_purple,
        ),
        # strokeDash=alt.StrokeDash("type").scale(range=line_strokeDash),
        tooltip=["year", "type", "occupancy rate"],
    )
)
occupancy_chart.save("output/Links/accommodation-occupancy.png", scale_factor=3)
occupancy_chart.properties(width="container").save(
    "output/Links/accommodation-occupancy.html"
)
occupancy_chart


In [ ]:
supplydemand_chart = (
    alt.Chart(
        supplydemand.with_columns(date=pl.date(pl.col("year"), 1, 1)).sort("year")
    )
    .mark_line()
    .encode(
        x=alt.X("date", title="year"),
        y=alt.Y("nights", title="SF accommodation supply/demand (nights)"),
        color=alt.Color("type").scale(
            # domain=["supply", "demand"],
            range=color_line_yellow_purple,
        ),
        strokeDash=alt.StrokeDash("supply/demand").scale(
            domain=["supply", "demand"], range=line_strokeDash
        ),
        tooltip=["year", "type", "supply/demand", "nights"],
    )
)
supplydemand_chart.save("output/Links/accommodation-supplydemand.png", scale_factor=3)
supplydemand_chart.properties(width="container").save(
    "output/Links/accommodation-supplydemand.html"
)
supplydemand_chart.interactive()
